In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import cv2
from preprocess import preprocessing
#from read_mias import *
import config
from object_detector import ObjectDetector
from custom_tensor_dataset import CustomTensorDataset
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from torch.nn import MSELoss
from torch.optim import Adam
from torchvision.models import resnet50
from sklearn.model_selection import train_test_split
from imutils import paths
import pickle
import torch
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.retinanet import RetinaNet, RetinaNet_ResNet50_FPN_V2_Weights
import albumentations as A
from albumentations.pytorch import ToTensorV2
from custom_tensor_dataset import CustomTensorDataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights
import random


https://debuggercafe.com/train-pytorch-retinanet-on-custom-dataset/#download-code



In [2]:
dir = os.getcwd()
info = pd.read_csv(dir +'/mias_info/labels.txt', sep = ' ', header = None)
info.columns = ['refno', 'tissue', 'class', 'severity','x','y','r']
info['severity'] = info['severity'].fillna('N')
info['image_id'] = info.refno.str.split('b', expand = True)[1].astype(int) -1
info.head()

,refno,tissue,class,severity,x,y,r,image_id
0,mdb001,G,CIRC,B,535,425,197.0,0
1,mdb002,G,CIRC,B,522,280,69.0,1
2,mdb003,D,NORM,N,NaN,NaN,NaN,2
3,mdb004,D,NORM,N,NaN,NaN,NaN,3
4,mdb005,F,CIRC,B,477,133,30.0,4


In [3]:
bm = info[info['severity'] != 'N']['refno']
notnormal = bm.tolist()

for i, name in enumerate(notnormal):
    notnormal[i] = name + '.pgm'

In [4]:
data_transform = A.Compose([
	A.Resize(512, 512),
	A.HorizontalFlip(p=0.5),
	A.Rotate(limit=15, p=1.0),
	#A.RandomBrightnessContrast(p=0.5),
	ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))


In [5]:
from custom_tensor_dataset import list_files_walk
from retinaNetDataset import customDataset


#all_imgs = list_files_walk('mias_data/')

train, test = train_test_split(notnormal, test_size = 0.2, random_state = 42)

train_dataset = customDataset(imgs = train, transforms=data_transform)
test_dataset = customDataset(imgs = test, transforms = data_transform)

def collate_fn(batch):
    return tuple(zip(*batch))

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size = config.BATCH_SIZE,
                                          shuffle = True,
                                          collate_fn = collate_fn)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size = config.BATCH_SIZE,
                                          shuffle = False,
                                          collate_fn = collate_fn)

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
for imgs, annotations in train_dataloader:
    imgs = list(img.to(device) for img in imgs)
    print(imgs[0].dtype)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)
    break

cpu
torch.float32
[{'boxes': tensor([[162.5490, 396.5586, 180.8593, 414.8690]]), 'labels': tensor([2]), 'image_id': tensor([31]), 'area': tensor([0.0011])}, {'boxes': tensor([[275.6486, 109.5401, 317.4865, 151.3780]]), 'labels': tensor([2]), 'image_id': tensor([13]), 'area': tensor([0.0052])}, {'boxes': tensor([[247.8227, 233.6779, 296.2085, 282.0637],
        [254.7294, 159.9907, 319.2437, 224.5051]]), 'labels': tensor([2, 2]), 'image_id': tensor([76]), 'area': tensor([0.0088, 0.0156])}, {'boxes': tensor([[245.9360, 262.9336, 293.8948, 310.8924]]), 'labels': tensor([1]), 'image_id': tensor([22]), 'area': tensor([0.0061])}]


In [7]:
from retina_model import create_model
import config_retina

model = create_model(config_retina.NUM_CLASSES)

print(model)


RetinaNet(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      

In [8]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
num_epochs = 20
#config_retina.NUM_EPOCHS
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

len_dataloader = len(train_dataloader)

train_losses = []  # Store epoch losses
val_loss = []

map_list = []
map_50_list = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    
    for imgs, annotations in train_dataloader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model([imgs[0]], [annotations[0]])
        losses = sum(loss for loss in loss_dict.values())        

        optimizer.zero_grad()
        losses.backward()
        optimizer.step() 

        epoch_loss += losses.item()  # Make sure to use .item() to extract scalar from tensor
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")
    train_losses.append(epoch_loss)

    # validation
    model.eval()
    test_loss = 0
    target = []
    preds = []      
    for imgs, annotations in test_dataloader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        with torch.no_grad():
            outputs = model(imgs)
            
        #losses = sum(loss for loss in outputs.values()) 
        #test_loss += losses.item()

        for i in range(len(imgs)):
            true_dict = dict()
            preds_dict = dict()
            true_dict['boxes'] = annotations[i]['boxes'].detach().cpu()
            true_dict['labels'] = annotations[i]['labels'].detach().cpu()
            preds_dict['boxes'] = outputs[i]['boxes'].detach().cpu()
            preds_dict['scores'] = outputs[i]['scores'].detach().cpu()
            preds_dict['labels'] = outputs[i]['labels'].detach().cpu()
            preds.append(preds_dict)
            target.append(true_dict)

    metric = MeanAveragePrecision()
    metric.update(preds, target)
    metric_summary = metric.compute()

    map_list.append(metric_summary['map'])
    map_50_list.append(metric_summary['map_50'])

    print(f'\t mAP: {metric_summary['map']}')
    print(f'\t mAP50: {metric_summary['map_50']}')
    
    val_loss.append(test_loss)

Epoch 1/20, Loss: 50.9260059595108


c:\Users\Student\bioe486\envs\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)


	 mAP: 0.0
	 mAP50: 0.0
Epoch 2/20, Loss: 43.8944525718689


c:\Users\Student\bioe486\envs\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)


	 mAP: 0.0
	 mAP50: 0.0
Epoch 3/20, Loss: 35.27771782875061


...

c:\Users\Student\bioe486\envs\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)


	 mAP: 0.006059448234736919
	 mAP50: 0.02223810739815235
Epoch 14/20, Loss: 29.956028163433075


c:\Users\Student\bioe486\envs\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)


	 mAP: 0.00010684521839721128
	 mAP50: 0.00017807536642067134


KeyboardInterrupt: 

In [ ]:
for imgs, annotations in test_dataloader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        break

In [ ]:
model.eval()
preds = model(imgs)
preds

In [ ]:
from plotting import plot_image
idx = 3
fig, axs = plt.subplots(1,2)
axs[0] = plot_image(imgs[idx], preds[idx])
axs[0].set_title('Prediction')
axs[1] = plot_image(imgs[idx], annotations[idx])